## Running example from the readme


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

def generate_gp_samples_with_library(n_groups, n_observations_per_group, n_dimensions, time_points, length_scale=1.0):
    """
    Generate multidimensional GP samples using scikit-learn.
    Each dimension has the same underlying process.
    """
    kernel = RBF(length_scale=length_scale)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-8)

    gp_sample_constant = gp.sample_y(time_points.reshape(-1, 1), random_state=None).flatten()

    samples = []
    for _ in range(n_groups):
        group_samples = []
        for _ in tqdm(range(n_observations_per_group)):
            # Generate independent samples for each dimension
            gp_samples_per_dim = []
            for _ in range(n_dimensions):
                gp_sample = gp.sample_y(time_points.reshape(-1, 1), random_state=None).flatten()
                gp_samples_per_dim.append(gp_sample)
            # Stack the samples to form a multidimensional sample
            gp_samples_multidim = np.stack(gp_samples_per_dim, axis=-1)
            group_samples.append(gp_samples_multidim)
        samples.append(np.array(group_samples))

    return samples

# Parameters
n_groups = 1
n_observations_per_group = 2000
n_dimensions = 3
n_time_points = 50
time_points = np.linspace(0, 10, n_time_points)
length_scale = 1.5

# Generate GP samples
gp_samples = generate_gp_samples_with_library(
    n_groups=n_groups,
    n_observations_per_group=n_observations_per_group,
    n_dimensions=n_dimensions,
    time_points=time_points,
    length_scale=length_scale
)

if False:
    # Plot example
    for dim in range(n_dimensions):
        for group_idx in range(n_groups):
            plt.figure()
            for observation in gp_samples[group_idx]:
                plt.plot(time_points, observation[:, dim], label=f"Dim {dim + 1}")
            plt.title(f"Group {group_idx + 1} Observations for Dimension {dim + 1}")
            plt.xlabel("Time")
            plt.ylabel("Value")
            plt.legend()
            plt.show()


100%|██████████████████████████████████████| 2000/2000 [00:03<00:00, 565.32it/s]


In [4]:
n_new_dims = 10
Proj = np.random.uniform(size = (n_dimensions, n_new_dims))#.reshape(1,1,n_dimensions, n_new_dims)
X_ori = gp_samples[0] @ Proj

In [5]:
list_p_dataset = [0.01, 0.3, 0.5]
list_p_model = [0.01, 0.05, 0.1, 0.3, 0.5, 0.8]
list_gamma = [0., 0.1, 0.5]

In [6]:
# Normalize

X = (X_ori - X_ori.mean(axis=(0,1)).reshape(1,1,-1)) / X_ori.std(axis=(0,1)).reshape(1,1,-1)

In [7]:
from pygrinder import mcar
X = mcar(X, 0.3)  # randomly hold out 10% observed values as ground truth
X[X!=X] = 0
dataset = {'X' : X}

In [8]:
print(f"numpy version: {np.__version__}")


numpy version: 1.24.4


In [9]:
#!pip install -r requirements/requirements.txt

In [23]:

# Model training. This is PyPOTS showtime.
from pypots.imputation import GP_VAE
from pypots.utils.metrics import calc_mae
from pypots.optim.adam import Adam
from torch.optim.lr_scheduler import LRScheduler

gpvae = GP_VAE(n_steps = dataset['X'].shape[1], 
            n_features = dataset['X'].shape[2], 
            latent_size = 3, 
            epochs = 200, 
            batch_size = 32,
            beta = .01, 
            K = 10,  
            encoder_sizes = (128,128), 
            decoder_sizes = (128,),
            optimizer = Adam(lr = 1e-4) #, v
            )

gpvae.model.backbone.gamma = 0.5
gpvae.model.backbone.p = 0.05
# Here I use the whole dataset as the training set because ground truth is not visible to the model, you can also split it into train/val/test sets
#with torch.autograd.set_detect_anomaly(False):
gpvae.fit(dataset)  # train the model on the dataset

2024-11-22 20:51:24 [INFO]: No given device, using default device: cpu
2024-11-22 20:51:24 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-11-22 20:51:24 [INFO]: GP_VAE initialized with the given hyperparameters, the number of trainable parameters: 21,782


Model dimensions is: 
GpvaeEncoder(
  (net): Sequential(
    (0): Linear(in_features=20, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
  (mu_layer): Linear(in_features=128, out_features=3, bias=True)
  (logvar_layer): Linear(in_features=128, out_features=3, bias=True)
)
GpvaeDecoder(
  (net): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


2024-11-22 20:51:30 [INFO]: Epoch 001 - training loss: 13484.0329


mean of mask of imputed coords : tensor(0.3027)


2024-11-22 20:51:37 [INFO]: Epoch 002 - training loss: 9080.6991
2024-11-22 20:51:42 [INFO]: Epoch 003 - training loss: 4499.9304


...

2024-11-22 20:59:49 [INFO]: Epoch 040 - training loss: 118.5083
2024-11-22 20:59:54 [INFO]: Epoch 041 - training loss: 117.1647


mean of mask of imputed coords : tensor(0.2962)


2024-11-22 21:00:00 [INFO]: Epoch 042 - training loss: 116.0584


mean of mask of imputed coords : tensor(0.2994)


2024-11-22 21:00:06 [INFO]: Epoch 043 - training loss: 114.9804
2024-11-22 21:00:11 [INFO]: Epoch 044 - training loss: 113.9338
2024-11-22 21:00:11 [WARNING]: ‼️ Training got interrupted by the user. Exist now ...
2024-11-22 21:00:11 [INFO]: Finished training. The best model is from epoch#44.


In [27]:
gpvae.fit(dataset)  # train the model on the dataset

2024-11-22 21:19:59 [INFO]: Epoch 001 - training loss: 12.9385


mean of mask of imputed coords : tensor(0.3039)


2024-11-22 21:20:04 [INFO]: Epoch 002 - training loss: 12.9792
2024-11-22 21:20:09 [INFO]: Epoch 003 - training loss: 12.9082


mean of mask of imputed coords : tensor(0.2991)


2024-11-22 21:20:14 [INFO]: Epoch 004 - training loss: 12.9431


...

2024-11-22 21:36:21 [INFO]: Epoch 197 - training loss: 10.1297
2024-11-22 21:36:25 [INFO]: Epoch 198 - training loss: 10.2173


mean of mask of imputed coords : tensor(0.3004)


2024-11-22 21:36:30 [INFO]: Epoch 199 - training loss: 10.1714


mean of mask of imputed coords : tensor(0.3016)


2024-11-22 21:36:35 [INFO]: Epoch 200 - training loss: 10.1497
2024-11-22 21:36:35 [INFO]: Finished training. The best model is from epoch#195.


In [28]:
## define tests to evaluate convergence of AE

## Assess reconstruction error
import torch

def get_errors(gpvae, X):

    X_corrupted = mcar(X, 0.3)
    X_corrupted[X_corrupted!=X_corrupted] = 0
    
    z = gpvae.model.backbone.encoder(torch.tensor(X).float()).mean
    X_recon = gpvae.model.backbone.decoder(z).mean
    
    z_corrupted = gpvae.model.backbone.encoder(torch.tensor(X_corrupted).float()).rsample()#sample_shape = torch.Size([10])
    X_recon_corrupted = gpvae.model.backbone.decoder(z_corrupted).mean
    
    error = (torch.tensor(X).float() - X_recon).pow(2)[X!=0].mean().item()
    
    error_corrupted = (torch.tensor(X).float() - X_recon_corrupted).pow(2)[X_corrupted!=0].mean().item()
    
    error_imputation_corrupted = (torch.tensor(X).float() - X_recon_corrupted).pow(2)[X_corrupted==0].mean().item()
    
    print(error, error_corrupted, error_imputation_corrupted)

    return [error, error_corrupted, error_imputation_corrupted]

err_p030 = get_errors(gpvae, X)


0.0007944988319650292 0.012348038144409657 0.586040198802948


In [17]:
errs = {}

errs[f'p_dataset_{p_dataset}_p_model_{p_model}_gamma_{gamma}'] = get_errors(gpvae, X)

0.001754634315147996 0.013832245022058487 0.5730319619178772


NameError: name 'p_dataset' is not defined

In [ ]:
gpvae.model.backbone.p = 0.1

#gpvae.model.backbone.load_state_dict(torch.load('model_weights2.pth'))
gpvae.epochs = 100
gpvae.fit(dataset)  # train the model on the dataset

2024-11-22 21:40:21 [INFO]: Epoch 001 - training loss: 15.8823


mean of mask of imputed coords : tensor(0.5144)


2024-11-22 21:40:26 [INFO]: Epoch 002 - training loss: 15.8925
2024-11-22 21:40:30 [INFO]: Epoch 003 - training loss: 15.7782


mean of mask of imputed coords : tensor(0.5126)


2024-11-22 21:40:36 [INFO]: Epoch 004 - training loss: 15.7673


...

2024-11-22 21:42:40 [INFO]: Epoch 029 - training loss: 14.8655
2024-11-22 21:42:45 [INFO]: Epoch 030 - training loss: 14.8799


mean of mask of imputed coords : tensor(0.5198)


2024-11-22 21:42:51 [INFO]: Epoch 031 - training loss: 14.8548


mean of mask of imputed coords : tensor(0.5144)


2024-11-22 21:42:58 [INFO]: Epoch 032 - training loss: 14.8573
2024-11-22 21:43:04 [INFO]: Epoch 033 - training loss: 14.7630


In [ ]:
# Assume 'model' is your neural network instance
torch.save(gpvae.model.backbone.state_dict(), 'model_weights_p030.pth')

#gpvae_backup = gpvae

In [ ]:
import torch
#gpvae.model.backbone.load_state_dict(torch.load('model_weights_p005.pth'))

gpvae.model.backbone.train_decoder = False
for param in gpvae.model.backbone.decoder.parameters():
    param.requires_grad = False
gpvae.epochs = 100
    
gpvae.model.backbone.p = 0.1
gpvae.model.backbone.gamma = 0.3

# Here I use the whole dataset as the training set because ground truth is not visible to the model, you can also split it into train/val/test sets
gpvae.fit(dataset)  # train the model on the dataset

In [ ]:
# Assume 'model' is your neural network instance
torch.save(gpvae.model.backbone.state_dict(), 'model_weights2.pth')

#gpvae_backup = gpvae

In [ ]:
gpvae.model.backbone.load_state_dict(torch.load('model_weights2.pth'))


for param in gpvae.model.backbone.decoder.parameters():
    param.requires_grad = False
    
gpvae.model.backbone.p = 0.3
# Here I use the whole dataset as the training set because ground truth is not visible to the model, you can also split it into train/val/test sets
gpvae.fit(dataset)  # train the model on the dataset

In [ ]:
#gpvae.model.backbone.load_state_dict(torch.load('model_weights2.pth'))


for param in gpvae.model.backbone.decoder.parameters():
    param.requires_grad = False
    
gpvae.model.backbone.p = 0.3
# Here I use the whole dataset as the training set because ground truth is not visible to the model, you can also split it into train/val/test sets
gpvae.fit(dataset)  # train the model on the dataset

## RUN THIS

In [ ]:
gpvae.model.backbone.p = 0.1

gpvae.fit(dataset)  # train the model on the dataset

In [ ]:
## Assess reconstruction error
import torch

z = gpvae.model.backbone.encoder(torch.tensor(X).float()).mean
X_recon = gpvae.model.backbone.decoder(z).mean

error = (torch.tensor(X).float() - X_recon).pow(2)[X!=0].mean()

print(error)

In [ ]:
for param in gpvae.model.backbone.decoder.parameters():
    param.requires_grad = False
    
gpvae.model.backbone.p = 0.1
# Here I use the whole dataset as the training set because ground truth is not visible to the model, you can also split it into train/val/test sets
gpvae.fit(dataset)  # train the model on the dataset

In [ ]:
gpvae.model.backbone.p = .3
gpvae.model.backbone.sampling = True
gpvae.fit(dataset)  # train the model on the dataset

In [ ]:
import gpytorch
import torch
with gpytorch.settings.cholesky_jitter(1e-4):
    with torch.autograd.set_detect_anomaly(True):
        gpvae.fit_kernel(dataset)  # train the model on the dataset


In [ ]:
gpvae.gp.gp_models[0].covar_module.outputscale


In [ ]:
help(gpvae.gp.gp_models[0].covar_module)


In [ ]:
import torch
x = torch.ones((8,48))
gpvae.gp.gp_models[0](x)

In [ ]:
gpvae.gp._fit_kernel

In [ ]:
imputation = gpvae.impute(dataset)  # impute the originally-missing values and artificially-missing values
indicating_mask = np.isnan(X) ^ np.isnan(X_ori)  # indicating mask for imputation error calculation
mae = calc_mae(imputation, np.nan_to_num(X_ori), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)
gpvae.save("save_it_here/gpvae_physionet2012.pypots")  # save the model for future use
gpvae.load("save_it_here/gpvae_physionet2012.pypots")  # reload the serialized model file for following imputation or training

In [ ]:

# Model training. This is PyPOTS showtime.
from pypots.imputation import GPVAE
from pypots.utils.metrics import calc_mae
gpvae = GPVAE(n_steps=48, n_features=37, latent_size = 12, epochs=10)
# Here I use the whole dataset as the training set because ground truth is not visible to the model, you can also split it into train/val/test sets
gpvae.fit(dataset)  # train the model on the dataset
imputation = gpvae.impute(dataset)  # impute the originally-missing values and artificially-missing values
indicating_mask = np.isnan(X) ^ np.isnan(X_ori)  # indicating mask for imputation error calculation
mae = calc_mae(imputation, np.nan_to_num(X_ori), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)
gpvae.save("save_it_here/gpvae_physionet2012.pypots")  # save the model for future use
gpvae.load("save_it_here/gpvae_physionet2012.pypots")  # reload the serialized model file for following imputation or training

In [ ]:

# Model training. This is PyPOTS showtime.
from pypots.imputation import SAITS
from pypots.utils.metrics import calc_mae
gpvae = SAITS(n_steps=48, n_features=37, n_layers=2, d_model=256, n_heads=4, d_k=64, d_v=64, d_ffn=128, dropout=0.1, epochs=10)
# Here I use the whole dataset as the training set because ground truth is not visible to the model, you can also split it into train/val/test sets
gpvae.fit(dataset)  # train the model on the dataset
imputation = gpvae.impute(dataset)  # impute the originally-missing values and artificially-missing values
indicating_mask = np.isnan(X) ^ np.isnan(X_ori)  # indicating mask for imputation error calculation
mae = calc_mae(imputation, np.nan_to_num(X_ori), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)
gpvae.save("save_it_here/saits_physionet2012.pypots")  # save the model for future use
gpvae.load("save_it_here/saits_physionet2012.pypots")  # reload the serialized model file for following imputation or training